# Proyecto Fin de Máster - Máster en Big Data y Business Analytics V Edición
# Big Data International Campus - Universidad Europea Miguel de Cervantes

### *Eliasib Jesús García Martín*
---

# 1. Introducción

El propósito de este proyecto es el de crear un producto de datos que sigan las metodologías de análisis de datos que se han impartido a lo largo de este máster. 

Para ello, se ha propuesto como conjunto de datos (en adelante, dataset) sobre los que aplicar dicho análisis el conjunto sobre datos de uso de un sistema de alquiler de bicicletas en el área de la Bahía de San Francisco (SF Bay Area Bike Share - https://www.kaggle.com/benhamner/sf-bay-area-bike-share).

En cuanto al análisis a realizar, se centrará en desarrollar, utilizando las metodologías más adecuadas, los siguientes puntos clave:
* Investigar la influencia del tiempo atmosférico en los viajes en bicicletas registrados en el dataset.
* Reconocer las variaciones en los patrones de los viajes registrados en función del momento del día y de los días de la semana en los que se producen.
* **/\*  TODO: Encontrar una comercialización para el modelo predictivo. \*/**

# 2. Carga de los datos y análisis descriptivo

El dataset sobre el que se basa este proyecto está compuesto por cuatro registros de datos, en forma de ficheros CSV (Comma Separated Values). La información contenida en estos cuatro ficheros está interrelacionada, por lo que la unión de todos ellos conforma el conjunto completo de los datos.

Los ficheros de los que se compone el dataset son:
* **station.csv:** En este fichero se listan las estaciones donde los usuarios pueden alquilar y/o depositar las bicicletas. Las columas para cada registro son:
    * **id** - Identificador del registro
    * **name** -  Nombre de la estación
    * **lat** - Latitud de la ubicación de la estación
    * **lon** - Longitud de la ubicación de la estación
    * **dock_count** - Número de plazas de bicicletas que puede alojar la estación
    * **city** - Ciudad donde se encuentra la estación
    * **installation_date** - Fecha de instalación de la estación


* **weather.csv:** En este fichero se listan los datos del tiempo atmosférico registrados diariamente. Las columnas para cada registro son:
    * **date** - Fecha del registro
    * **max_temperature_f** - Temperatura máxima en grados Fahrenheit
    * **mean_temperature_f** - Temperatura media en grados Fahrenheit
    * **min_temperature_f** - Temperatura mínima en grados Fahrenheit
    * **max_dew_point_f** - Temperatura máxima de rocío en grados Fahrenheit
    * **mean_dew_point_f** - Temperatura media de rocío en grados Fahrenheit
    * **min_dew_point_f** - Temperatura mínima de rocío en grados Fahrenheit
    * **max_humidity** - Humedad máxima
    * **mean_humidity** - Humedad media
    * **min_humidity** - Humedad mínima
    * **max_sea_level_pressure_inches** - Presión atmosférica máxima a nivel del mar en pulgadas de mercurio
    * **mean_sea_level_pressure_inches** - Presión atmosférica media a nivel del mar en pulgadas de mercurio
    * **min_sea_level_pressure_inches** - Presión atmosférica mínima a nivel del mar en pulgadas de mercurio
    * **max_visibility_miles** - Visibilidad máxima en millas
    * **mean_visibility_miles** - Visibilidad media en millas
    * **min_visibility_miles** - Visibilidad mínima en millas
    * **max_wind_speed_mph** - Velocidad máxima del viento en millas por hora
    * **mean_wind_speed_mph** - Velocidad media del viento en millas por hora
    * **min_wind_speed_mph** - Velocidad mínima del viento en millas por hora
    * **precipitation_inches** - Precipitaciones en pulgadas
    * **cloud_cover** - Nubosidad en octas
    * **events** - Eventos atmosféricos (lluvia, niebla, tormenta...)
    * **wind_dir_degrees** - Dirección del viento en grados
    * **zip_code** - Código postal de la localización del registro


* **trip.csv:** En este fichero se listan los viajes en bicicleta registrados. Las columnas para cada registro son:
    * **id** - Identificador del registro
    * **duration** - Duración del viaje en segundos
    * **start_date** - Fecha y hora de inicio del viaje
    * **start_station_name** - Nombre de la estación donde se inició el viaje
    * **start_station_id** - Identificador de la estación donde se inició el viaje
    * **end_date** - Fecha y hora de finalización del viaje
    * **end_station_name** - Nombre de la estación donde se finalizó el viaje
    * **end_station_id** - Identificador de la estación donde se finalizó el viaje
    * **bike_id** - Identificador de la bicicleta utilizada para el viaje


* **status.csv:** En este fichero se listan registros por minuto del estado de las estaciones. Las columnas para cada registro son:
    * **station_id** - Identificador de la estación a la que se refiere el registro
    * **bikes_available** - Número de bicicletas disponibles
    * **docks_available** - Número de plazas de estacionamiento para bicicletas disponibles
    * **time** - Fecha y hora del registro
    

